In [2]:
# Code for the solution of master problem. Inputs will be generated line of flight, costs of assignment of aircrafts
# those line of flights. Output should be dual variables and printing of the total cost.
import sys
import docplex
from docplex.mp.model import Model
import numpy as np

In [3]:
def MasterProb(Pos_lof_c, flight_obj_dict, airport_obj_dict, can_cos=28800):
    # Create model object
    mdl = Model(name ="mdl")
    
    #Definition of decision variables
    dims=[]
    for a in list(Pos_lof_c.keys()):
        for l in Pos_lof_c[a]:
            dims.append((a,l))
    x   =   mdl.continuous_var_dict(dims, lb=0, name="x")
    y   =   mdl.continuous_var_dict(flight_obj_dict.keys(),lb=0, name="y")
    
    # Add constraints for the problem
    #Constraint 1 :- A maximum of 1 lof can be assigned to an airoplane
    c1=[]
    c2=[]
    c3=[]
    for a in list(Pos_lof_c.keys()):
        c2.append(mdl.sum(x[(a, l)] for l in Pos_lof_c[a]) <= 1) 
        mdl.add(c2[-1])
    #Constraint 2 :- A flight is covered by maximum 1 assigned lof or it is cancelled 
    for f in list(flight_obj_dict.keys()):
        c1.append(mdl.sum(x[(a,l)]*l.is_f(flight_obj_dict[f]) for a in list(Pos_lof_c.keys()) for l in Pos_lof_c[a]) + y[f]  == 1)
        mdl.add(c1[-1])
    #Constraint 3:-  Every airport has airoplane at the end
    for p in list(airport_obj_dict.keys()):
        c3.append(mdl.sum(x[(a,l)]*l.is_p(p) for a in list(Pos_lof_c.keys()) for l in Pos_lof_c[a])  == airport_obj_dict[p])
        mdl.add(c3[-1])
    
    mdl.minimize(mdl.sum(x[(c,l)]*Pos_lof_c[c][Pos_lof_c[c].index(l)].cost for (c,l) in dims) + mdl.sum(y[f]*can_cos for f in flight_obj_dict.keys()) )
    
    msol = mdl.solve()
    print(msol.get_objective_value())
    for a in list(Pos_lof_c.keys()):
        for l in Pos_lof_c[a]:
            if msol[x[(a,l)]]>0:
                print("Aircraft ",a," lof ",Pos_lof_c[a].index(l),"= ",msol[x[(a,l)]], end=" ")
    dual1 = mdl.dual_values(c1)
    dual2 = mdl.dual_values(c2)
    dual3 = mdl.dual_values(c3)
    return dual1, dual2, dual3

In [2]:
def MasterProbFinal(Pos_lof_c, flight_obj_dict, airport_obj_dict, can_cos=28800):
    
    # Create model object
    mdl = Model(name ="mdl")
    
    #Definition of decision variables
    dims=[]
    for a in list(Pos_lof_c.keys()):
        for l in Pos_lof_c[a]:
            dims.append((a,l))
    x   =   mdl.binary_var_dict(dims, lb=0, name="x")
    y   =   mdl.binary_var_dict(flight_obj_dict.keys(),lb=0, name="y")
    
    # Add constraints for the problem
    #Constraint 1 :- A maximum of 1 lof can be assigned to an airoplane
    c1=[]
    c2=[]
    c3=[]
    for a in list(Pos_lof_c.keys()):
        c2.append(mdl.sum(x[(a, l)] for l in Pos_lof_c[a]) <= 1) 
        mdl.add(c2[-1])
    #Constraint 2 :- A flight is covered by maximum 1 assigned lof or it is cancelled 
    for f in list(flight_obj_dict.keys()):
        c1.append(mdl.sum(x[(a,l)]*l.is_f(flight_obj_dict[f]) for a in list(Pos_lof_c.keys()) for l in Pos_lof_c[a]) + y[f]  == 1)
        mdl.add(c1[-1])
    #Constraint 3:-  Every airport has airoplane at the end
    for p in list(airport_obj_dict.keys()):
        c3.append(mdl.sum(x[(a,l)]*l.is_p(p) for a in list(Pos_lof_c.keys()) for l in Pos_lof_c[a])  == airport_obj_dict[p])
        mdl.add(c3[-1])
    
    mdl.minimize(mdl.sum(x[(c,l)]*Pos_lof_c[c][Pos_lof_c[c].index(l)].cost for (c,l) in dims) + mdl.sum(y[f]*can_cos for f in flight_obj_dict.keys()) )
    
    msol = mdl.solve()
    print(msol.get_objective_value())
    file_o = open("Output.txt", "w")
    for a in list(Pos_lof_c.keys()):
        for l in Pos_lof_c[a]:
            if msol[x[(a,l)]]>0:
                print("Aircraft ",a," lof ",Pos_lof_c[a].index(l),"= ",msol[x[(a,l)]])
                l.display()
                file_o.write("\n"+a+"---Route"+"\n")
                file_o.write(l.info)
    for f in flight_obj_dict.keys():
        if msol[y[f]]==1:
            file_o.write("\n"+str(f)+"---Cancelled")
    file_o.close()
    #dual1 = mdl.dual_values(c1)
    #dual2 = mdl.dual_values(c2)
    #dual3 = mdl.dual_values(c3)
    #return dual1, dual2, dual3